In [ ]:
import os
import numpy as np
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split
import logging
from colorama import Fore, Style, init
import warnings
import argparse

# filter warnings
warnings.filterwarnings('ignore')

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO, format=f'{Fore.GREEN}%(asctime)s - %(levelname)s - %(message)s{Style.RESET_ALL}')

# Initialize colorama
init(autoreset=True)

global_path = '../data'
triplet_path = f"{global_path}/train_triplets.txt"
unique_tracks_path = f"{global_path}/p02_unique_tracks.txt"
genre_path = f"{global_path}/p02_msd_tagtraum_cd2.cls"

In [ ]:
def load_data(triplet_path, unique_tracks_path):
    logging.info('Loading data...')


    triplet_columns = ['user_id', 'song_id', 'play_count']
    track_columns = ['track_id', 'song_id', 'artist', 'title']

    triplet_df = pl.read_csv(triplet_path, separator='\t', new_columns=triplet_columns, use_pyarrow=True)
    unique_tracks_df = pl.read_csv(unique_tracks_path, new_columns=track_columns, use_pyarrow=True)

    logging.info('Data loaded successfully.')

    logging.info('Merging songs...')

    triplet_df = triplet_df.filter(pl.col('play_count') > 1)
    songs = pd.merge(triplet_df.to_pandas(), unique_tracks_df.to_pandas(), on='song_id', how='left')
    songs['song'] = songs['title']+' - ' + songs['artist']
    songs = songs[['user_id', 'song_id', 'track_id', 'song', 'play_count']]

    songs['user_idx'] = pd.factorize(songs['user_id'])[0]
    songs['song_idx'] = pd.factorize(songs['song_id'])[0]

    logging.info('Songs merged successfully.')

    del triplet_df, unique_tracks_df

    # save the data
    # songs.to_csv('data/songs.csv', index=False)

    return songs

In [ ]:
songs = load_data(triplet_path, unique_tracks_path)
X = songs[['user_idx', 'song_idx', 'song', 'play_count']]
X

In [ ]:
unique_songs_df = X[['song_idx', 'song']].drop_duplicates(subset='song_idx')
unique_songs_df

In [ ]:
# save two csv files, ratings.csv and songs.csv
unique_songs_df.to_csv(f"{global_path}/songs.csv", index=False)
songs[['user_idx', 'song_idx', 'play_count']].to_csv(f"{global_path}/ratings.csv", index=False)

In [ ]:
num_ratings = X.groupby('song_idx')['play_count'].count()
mean_rating = X.groupby('song_idx')['play_count'].mean()
sum_ratings = X.groupby('song_idx')['play_count'].sum()

In [ ]:
unique_songs_df['num_ratings'] = unique_songs_df['song_idx'].map(num_ratings)
unique_songs_df['mean_rating'] = unique_songs_df['song_idx'].map(mean_rating)

In [ ]:
damping_factor = 10
global_mean_rating = X['play_count'].mean()

global_mean_rating, damping_factor

In [ ]:
damped_numerator = sum_ratings + damping_factor * global_mean_rating
damped_denominator = num_ratings + damping_factor
damped_mean_rating = damped_numerator / damped_denominator
unique_songs_df['damped_mean_rating'] = unique_songs_df['song_idx'].map(damped_mean_rating)

In [ ]:
unique_songs_df.sort_values(by='num_ratings', ascending=False).head(10)

In [ ]:
unique_songs_df.sort_values(by='mean_rating', ascending=False).head(10)

In [ ]:
unique_songs_df.sort_values(by='damped_mean_rating', ascending=False).head(10)

## Using LLMs to recommend songs

## Load the data & data preparation

In [ ]:
def load_data(triplet_path, unique_tracks_path, genre_path):
    logging.info('Loading data...')
    triplet_columns = ['user_id', 'song_id', 'play_count']
    track_columns = ['track_id', 'song_id', 'artist', 'title']
    genre_column_names = ['track_id', 'majority_genre', 'minority_genre']

    triplet_df = pl.read_csv(triplet_path, separator='\t', new_columns=triplet_columns, use_pyarrow=True)
    unique_tracks_df = pl.from_pandas(pd.read_csv(unique_tracks_path, names=track_columns, sep="<SEP>", engine='python'))
    genre_df = pl.from_pandas(pd.read_csv(genre_path, sep='\t', comment='#', names=genre_column_names))

    logging.info('Data loaded successfully.')
    return triplet_df, unique_tracks_df, genre_df.drop(columns=['minority_genre'])

In [ ]:
triplet_df, unique_tracks_df, genre_df = load_data(triplet_path, unique_tracks_path, genre_path)
# songs = pd.read_csv(f"{global_path}/songs.csv")

In [ ]:
triplet_df

In [ ]:
song_play_counts = triplet_df.group_by('song_id').agg(pl.sum('play_count').alias('play_count'))
song_play_counts

In [ ]:
unique_tracks_df

In [ ]:
genre_df

In [ ]:
o = song_play_counts.join(unique_tracks_df, on='song_id')
o

In [ ]:
o = o.join(genre_df, on='track_id')
o

In [ ]:

# Assuming triplet_df, unique_tracks_df, and genre_df are already defined Polars DataFrames
# Merge triplet_df with unique_tracks_df
songs_df = triplet_df.join(
    unique_tracks_df,
    on="song_id",
    how="left"
)
songs_df

In [ ]:
# Merge songs_df with genre_df
full_songs_df = songs_df.join(
    genre_df,
    on="track_id",
    how="left"
)
full_songs_df

In [ ]:
full_songs_df.write_csv(f"{global_path}/full_songs.csv")

In [ ]:
def read_lyrics_file(file_path):
    lyrics_dataset = []
    with open(file_path, 'r', encoding='utf-8') as file:

        for line in file:
            if line.startswith("#") or not line.strip():
                continue
            elif line.startswith('%'):
                    word_list = line[1:].strip().split(',')
            else:
                parts = line.split(",")
                track_id = parts[0]
                word_counts = parts[2:]

                lyrics = []
                for wc in word_counts:
                    idx, count = map(int, wc.split(":"))
                    lyrics.extend([word_list[idx - 1]])  # Word index is 1-based

                lyrics_text = ' '.join(lyrics)
                lyrics_dataset.append((track_id, lyrics_text))

    return pl.DataFrame(lyrics_dataset, schema=['track_id', 'lyrics'])

# Usage example:
file_path = '../data/mxm_dataset_train.txt'
lyrics_df = read_lyrics_file(file_path)

In [ ]:
lyrics_df

In [ ]:
full_songs_df = pl.read_csv(f"{global_path}/full_songs.csv")

In [ ]:
full_songs_df = full_songs_df.join(
    lyrics_df,
    on="track_id",
    how="left"
)

full_songs_df

In [ ]:
o = o.join(lyrics_df, on='track_id')

In [ ]:
o.write_csv(f"{global_path}/llm_RecSys_dataset.csv")

In [ ]:
#Combine title, synopsis, and Genre
o = o.to_pandas()
o['combined_info'] = o.apply(lambda row: f"Song ID: {row['song_id']}\n Artist : {row['artist']}\n Title : {row['title']}\n Lyrics: {row['lyrics']}.\n Genres: {row['majority_genre']}", axis=1)
print(o['combined_info'][0])

In [ ]:
o[['combined_info']].to_csv(f"{global_path}/llm_RecSys_dataset_updated.csv", index=False)

In [ ]:
o.combined_info

## Build the LLM recommender.

In [ ]:
import os
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
loader = CSVLoader(file_path=f"{global_path}/llm_RecSys_dataset_updated.csv")
data = loader.load()

#data transformers
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

#Vector DB
docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
query = "I'm looking for a song similar by rapper like Eminem, 50 Cent and Snopp Dog. What could you suggest to me?"
docs = docsearch.similarity_search(query, k=1)

In [ ]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=api_key)
qa = RetrievalQA.from_chain_type(llm,
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True)

In [ ]:
result = qa.invoke({"query": query})
result['result']

In [ ]:
print(result['result'])

In [ ]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a music recommender system that helps users find songs that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three songs, with a short description of the song's genre, mood, and the reason why the user might like it.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'female')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

In [ ]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)
query = "I'm looking for a song similar by rapper like Eminem, 50 Cent. What could you suggest to me?"
result = qa({'query':query})
print(result['result'])

In [ ]:
result.get('source_documents')

In [ ]:
from langchain.prompts import PromptTemplate

template = """You are a music recommender system that helps users find songs that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three songs, with a short description of the song's genre, mood, and the reason why the user might like it.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.


{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=api_key) 

qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I'm looking for a song similar to pink floyd style. What could you suggest to me?"
result = qa.invoke({'query':query})
print(result['result'])

In [ ]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a music recommender system that helps users find songs that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three songs, with a short description of the song's genre, mood, and the reason why the user might like it.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'female')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

In [ ]:
PROMPT = PromptTemplate(template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)
PROMPT

In [ ]:
query = "I'm looking for rap songs, artists like eminem and 50cent. What could you suggest to me?"
result = qa.invoke({'query':query})
print(result['result'])
print(result['source_documents'])